see README for how I generated the first set of training data (via Prodigy)

this is how I'm generating training data from the RSS database.

In [1]:
from os import environ, makedirs
from os.path import join, exists, dirname
import requests
from urllib.parse import urlencode
import logging
import csv
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
from os import environ
from dotenv import load_dotenv
load_dotenv()

live_engine = create_engine(environ.get("LIVE_DATABASE_URL"))


/Users/jeremybmerrill/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [20]:
SEARCH_WARRANT_CSV_FN = 'search_warrants_model_training/search_warrants_for_ner_from_rss.csv'
with open(SEARCH_WARRANT_CSV_FN, 'w') as outcsv:
    writer = None
            
    sw_docs = pd.read_sql("""select case_name, min(document_type) document_type, case_number, court from rss_docket_entries where substring(case_number, 6, 2) = 'sw' group by case_name, case_number, court""", live_engine)
    for i, result in sw_docs.iterrows():
        if not writer:
            writer = csv.DictWriter(outcsv, fieldnames=result.to_dict().keys())
            writer.writeheader()
        writer.writerow(result.to_dict())

    csvs = [
            "search_warrants_model_training/search warrant training data - rss_document_types_all_courts.csv", 
    ]
    for csvfn in csvs:
        rows = pd.read_csv(csvfn)
        rows = rows[rows["is search warrant?"]]
        rows = rows[["case_name", "document_type", "case_number", "court"]]
        for i, result in rows.iterrows():
            if not writer:
                writer = csv.DictWriter(outcsv, fieldnames=result.to_dict().keys())
                writer.writeheader()
            writer.writerow(result.to_dict())



In [21]:
!grep 'USA v. Information' search_warrants_model_training/search_warrants_for_ner_from_rss.csv

what can we ignore from the starts of the training data?

In [2]:
import json
import re
examples = []
with open("search_warrants_model_training/ner_search_warrant_objects.train.json", 'r') as f:
    for line in f.readlines():
        examples.append(' '.join(json.loads(line)["tokens"]))


In [6]:
import re
from collections import Counter
Counter([re.split(r'as to', eg, 1)[0] for eg in examples]).most_common(10)


[('Application and Affidavit for Search and Seizure Warrant entered ', 53),
 ('Application for Search Warrant ', 26),
 ('Application & Affidavit for Search Warrant ', 23),
 ('APPLICATION and Affidavit for Search Warrant by USA ', 20),
 ('Application and Affidavit for a Search Warrant ', 16),
 ('Search and Seizure Warrant Issued in case ', 14),
 ('APPLICATION AND AFFIDAVIT for Search Warrant ', 14),
 ('APPLICATION AND AFFIDAVIT by USA ', 12),
 ('APPLICATION and Affidavit for Search Warrant ', 11),
 ('APPLICATION and Affidavit for Search Warrant - SEALED ', 8)]